# OOP - Inheritance

One of the most important notions of OOP is the **recycling of code**, and the most well-known example of this notion is the **inheritance**. Inheritance allows us to define a **child** of a specific **parent** class that "inherits" all the properties (attributes and methods) of the parent. The child class is usually used to implement a special case of the parent class, and they are also known as super-class and sub-class. It should be noted that a parent can have many children, and a children can have many parents.

The inheritance declaration is easy to implement, and it is done by specifying the parent class right in the child definition line. The child must have its own constructor (\_\__init_\_\__()_ method), but (usually) it will include relevant modifications of an instance of the parent, instantiated by calling the parent's constructor. 

For compatibility we repeat here _Table_'s implementation.

In [1]:
class Table:
    def __init__(self, *fields):
        self.fields = list(fields)
        self.n_fields = len(fields)
        self.records = []
        self.n_records = 0
    
    def __str__(self):
        template = "{:^15}" + (self.n_fields - 1) * "|{:^15}"
        if self.n_records < 10:
            records_str = "\n".join([template.format(*rec) for rec in self.records])
        else:
            records_str = "\n".join([template.format(*rec) for rec in self.records[:5]]) + "\n. . ."
            records_str += "\n".join([template.format(*rec) for rec in self.records[-5:]])

        header =  template.format(*self.fields) + "\n"
        horizontal_line = len(header) * "-" + "\n"
        return header + horizontal_line + records_str
    
    def __contains__(self, item):
        return item in self.get_column(self.fields[0])
    
    def __eq__(self, other):
        if self.fields[0] != other.fields[0]:
            return False
        return set(self.get_column(self.fields[0])) == set(other.get_column(other.fields[0]))
    
    def __ne__(self, other):
        return not (self == other)
    
    def __lt__(self, other):
        if self.fields[0] != other.fields[0]:
            return False
        return set(self.get_column(self.fields[0])) < set(other.get_column(other.fields[0]))
    
    def __gt__(self, other):
        if self.fields[0] != other.fields[0]:
            return False
        return set(self.get_column(self.fields[0])) > set(other.get_column(other.fields[0]))

    def __le__(self, other):
        return not (self > other)
    
    def __ge__(self, other):
        return not (self < other)
    
    def __add__(self, other):
        if self.fields != other.fields:
            return None
        ret = Table(*self.fields)
        for rec in self.records:
            ret.add_record(rec)
        keys1 = self.get_column(self.fields[0])
        for rec in other.records:
            if rec[0] not in keys1:
                ret.add_record(rec)
        return ret                
    
    def __getitem__(self, key):
        return self.get_records(self.fields[0], key).records[0]
    
    def add_record(self, rec):
        self.records.append(rec)
        self.n_records += 1
        
    def remove_record(self, rec):
        self.records.remove(rec)
        self.n_records -= 1
        
    def get_column(self, field):
        ind = self.fields.index(field)
        return [rec[ind] for rec in self.records]        
                
    def get_records(self, field, value):
        if field in self.fields:
            ind = self.fields.index(field)
            ret = Table(*self.fields)
            for rec in self.records:
                if rec[ind] == value:
                    ret.add_record(rec)
            return ret
        else:
            print("{} is not a field in the table.".format(field))
            
    def get_fields(self, *fields):
        if all([(field in self.fields) for field in fields]):
            columns = [self.get_column(field) for field in fields]
            records = [list(rec) for rec in zip(*columns)]
            ret = Table(*fields)
            for rec in records:
                ret.add_record(rec)
            return ret
        else:
            print("{} is not a field in the table.".format(field))

## Instantiation

Continuing the _Table_ example, we now define _SortedTable(Table)_. _SortedTable_ has all the properties of _Table_, but we also want its instances to keep their records sorted by the values of a specified argument _key_\__field_. Pay attention to the following when reviewing the _SortedTable_'s constructor given below:

1. The name of the parent(s) is given in parentheses right after the name of the child.
2. The \_\__init_\_\__()_ of the child includes a call to the \_\__init_\_\__()_ of the parent with a proper subset of itsarguments.
3. Since _self_ cannot "logically" call the \_\__init_\_\__()_ of its parent, then a special syntax is used, and it is called by the actual name of the parent class.
4. Sub-classes may (and usually do) have their own attributes, which not surprisingly is what differentiate them from their parents.

One additional note is that it is a good practice to name the class with the "CapWords" convention of upper- and lower-case.

In [2]:
class SortedTable(Table):                                       # (1)
    def __init__(self, key_field, *fields):                     # (2)
        Table.__init__(self, *fields)                           # (3)
        if key_field not in fields:
            print("ERROR!!! No such key. Please try again...")
        else:
            self.key_field = key_field                          # (4)

We note that an instance of a child is also an instance of the father. This may be confusing, because as we will see later, children may override their parent's methods.

In [3]:
customers = Table('name', 'address', 'age')  
print(isinstance(customers, SortedTable))
print(isinstance(customers, Table))

False
True


In [4]:
customers = SortedTable('address', 'name', 'address', 'age')  
print(isinstance(customers, SortedTable))
print(isinstance(customers, Table))

True
True


## Methods

Usually the child "inherits" a lot from its parent, so most of the parent's functionalities are still applicable without the need to rewrite them (hence the usefulness of inheritance). However, there is always a difference between them (otherwise, why bother creating the child?), expressed by either **new** or **overriding** methods of the child.

### Adding new methods

The most basic differentiation between a child and its parent are the properties that only the child has. We already saw that the attribute _key_\__field_ is defined only for _SortedTable_, and now we will implement a method that will also be available only for _SortedTable_ instances:
* _change_\__key(self, new_\__key_\__field)_ .

Bottom line - there is nothing special about defining new properties for a child class. It's exactly like we did it before.

In [5]:
class SortedTable(Table):
    def __init__(self, key_field, *fields):
        Table.__init__(self, *fields)
        if key_field not in fields:
            print("ERROR!!! The specified key is not defined. Table not created. Please try again...")
        else:
            self.key_field = key_field
        
    def change_key(self, new_key_field):
        # Modify the key
        self.key_field = new_key_field
        # Re-sorting the records
        key_index = self.fields.index(self.key_field)
        self.records = sorted(self.records, key=lambda x: x[key_index])

#### Demonstration

In [6]:
customers = SortedTable('address', 'name', 'address', 'age')
customers.add_record(['Russell Crowe', 'Dizengoff 4', 51])
customers.add_record(['Nicolas Cage', 'Basel 7', 52])
customers.add_record(['Gwyneth Paltrow', 'Weizmann 8', 43])
customers.add_record(['Al Pacino', 'Allenby 1', 63])
customers.add_record(['Diane Keaton', 'Basel 9', 52])

In [7]:
print(customers)

     name      |    address    |      age      
------------------------------------------------
 Russell Crowe |  Dizengoff 4  |      51       
 Nicolas Cage  |    Basel 7    |      52       
Gwyneth Paltrow|  Weizmann 8   |      43       
   Al Pacino   |   Allenby 1   |      63       
 Diane Keaton  |    Basel 9    |      52       


In [8]:
print(customers.key_field)

address


In [9]:
customers.change_key('name')
print(customers.key_field)

name


In [10]:
print(customers)

     name      |    address    |      age      
------------------------------------------------
   Al Pacino   |   Allenby 1   |      63       
 Diane Keaton  |    Basel 9    |      52       
Gwyneth Paltrow|  Weizmann 8   |      43       
 Nicolas Cage  |    Basel 7    |      52       
 Russell Crowe |  Dizengoff 4  |      51       


### Overriding parent's methods

Continuing with our example, we figure that the following methods we inherited from _Table_ are not influenced by the "sorting" feature: \_\__str_\_\__()_, Comparisons, \_\__add_\_\__()_, \_\__getitem_\_\__()_, _remove_\__record()_, _get_\__column()_, _get_\__records()_ and _get_\__fields()_.

Amazingly only one method should be overridden - _add_\__record()_. This method should be responsible for "inserting" the new record in the right place. We will write a method for the child with the **same name**, and since the child will always prefer its own method, this will practically override the parent's method.

We also note that there are two types of override: the first is to completely ignore the parent's method, and the second is to re-use the parent's method. We will demonstrate both approaches.

#### Approach 1 - complete override

In [11]:
class SortedTable(Table):
    def __init__(self, key_field, *fields):
        Table.__init__(self, *fields)
        if key_field not in fields:
            print("ERROR!!! The specified key is not defined. Table not created. Please try again...")
        else:
            self.key_field = key_field
        
    def change_key(self, new_key_field):
        # Modify the key
        self.key_field = new_key_field
        # Re-sorting the records
        key_index = self.fields.index(self.key_field)
        self.records = sorted(self.records, key=lambda x: x[key_index])
        
    def add_record(self, rec):
        key_index = self.fields.index(self.key_field)
        if self.n_records == 0:
            self.records.append(rec)
        elif rec[key_index] < self.records[0][key_index]:
            # rec is "smaller" than all records
            self.records.insert(0, rec)
        else:
            # Looking for the right index to "push" the record
            for ind in range(self.n_records):
                if rec[key_index] > self.records[ind][key_index]:
                    self.records.insert(ind+1, rec)
                    break
        self.n_records += 1

In [12]:
customers = SortedTable('name', 'name', 'address', 'age')
customers.add_record(['Russell Crowe', 'Dizengoff 4', 51])
customers.add_record(['Nicolas Cage', 'Basel 7', 52])
customers.add_record(['Gwyneth Paltrow', 'Weizmann 8', 43])
customers.add_record(['Al Pacino', 'Allenby 1', 63])
customers.add_record(['Diane Keaton', 'Basel 9', 52])
print(customers)

     name      |    address    |      age      
------------------------------------------------
   Al Pacino   |   Allenby 1   |      63       
 Diane Keaton  |    Basel 9    |      52       
Gwyneth Paltrow|  Weizmann 8   |      43       
 Nicolas Cage  |    Basel 7    |      52       
 Russell Crowe |  Dizengoff 4  |      51       


In [13]:
customers.change_key('age')
print(customers)

     name      |    address    |      age      
------------------------------------------------
Gwyneth Paltrow|  Weizmann 8   |      43       
 Russell Crowe |  Dizengoff 4  |      51       
 Diane Keaton  |    Basel 9    |      52       
 Nicolas Cage  |    Basel 7    |      52       
   Al Pacino   |   Allenby 1   |      63       


#### Approach 2 - re-using the parent's methods 

This method _add_\__record()_ works, but it doesn't "recycle" the father's code, which sometimes is a waste. A better implementation will call the father's methods where appropriate.

Note that like we saw with the \_\__init_\_\__()_ method (comment \#3 in the "Instantiation" paragraph above) the child's method is not defined yet, so we can't call _self_, hence we use the parent's class name and send _self_ as an input argument.

In [14]:
class SortedTable(Table):
    def __init__(self, key_field, *fields):
        Table.__init__(self, *fields)
        if key_field not in fields:
            print("ERROR!!! The specified key is not defined. Table not created. Please try again...")
        else:
            self.key_field = key_field
        
    def change_key(self, new_key_field):
        # Modify the key
        self.key_field = new_key_field
        # Re-sorting the records
        key_index = self.fields.index(self.key_field)
        self.records = sorted(self.records, key=lambda x: x[key_index])
        
    def add_record(self, rec):
        Table.add_record(self, rec)  # <-- This is the re-use
        key_index = self.fields.index(self.key_field)
        self.records = sorted(self.records, key=lambda x: x[key_index])

In [15]:
customers = SortedTable('name', 'name', 'address', 'age')
customers.add_record(['Russell Crowe', 'Dizengoff 4', 51])
print(customers)
customers.add_record(['Nicolas Cage', 'Basel 7', 52])
print(customers)
customers.add_record(['Gwyneth Paltrow', 'Weizmann 8', 43])
print(customers)
customers.add_record(['Al Pacino', 'Allenby 1', 63])
print(customers)
customers.add_record(['Diane Keaton', 'Basel 9', 52])
print(customers)

     name      |    address    |      age      
------------------------------------------------
 Russell Crowe |  Dizengoff 4  |      51       
     name      |    address    |      age      
------------------------------------------------
 Nicolas Cage  |    Basel 7    |      52       
 Russell Crowe |  Dizengoff 4  |      51       
     name      |    address    |      age      
------------------------------------------------
Gwyneth Paltrow|  Weizmann 8   |      43       
 Nicolas Cage  |    Basel 7    |      52       
 Russell Crowe |  Dizengoff 4  |      51       
     name      |    address    |      age      
------------------------------------------------
   Al Pacino   |   Allenby 1   |      63       
Gwyneth Paltrow|  Weizmann 8   |      43       
 Nicolas Cage  |    Basel 7    |      52       
 Russell Crowe |  Dizengoff 4  |      51       
     name      |    address    |      age      
------------------------------------------------
   Al Pacino   |   Allenby 1   |   

> **NOTE:** In this case, the second approach is less efficient, but it demonstrates the concept.